## DuckDB Python API
- the same way of loading sakila data as via terminal


In [1]:
import duckdb
from pathlib import Path # without pathlib you need to delete everything

### read data frame one by one

In [3]:
# create variable to store duckdb path
duckdb_path = "data/sakila.duckdb"
Path(duckdb_path).unlink(missing_ok=True) 

# connect to duckdb file with context manager,
# and ingest the data. 
# same as duckdb sakila.duckdb < sql/laod_sakila.sql
with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_scripts:
    conn.sql(ingest_scripts.read())
    # create pandas dataframes from the created duckdb connection
    description = conn.sql("DESC;").df()
    film = conn.sql("FROM film;").df()

# check the created dataframe
film.head(2)


,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00


In [4]:
description.head(3)

,database,schema,name,column_names,column_types,temporary
0,sakila,main,actor,"[actor_id, first_name, last_name, last_update]","[DOUBLE, VARCHAR, VARCHAR, TIMESTAMP]",False
1,sakila,main,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False
2,sakila,main,category,"[category_id, name, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",False


### read all tables into dataframes

In [5]:
# create an empty dictionary to keep all dataframes
dfs = {}

with duckdb.connect(duckdb_path) as conn: 
    for name in description["name"]:
        dfs[name] = conn.sql(f"FROM {name};").df() # to store all dataframes from description by column 'name' into dictionary
        



In [6]:
# check the resulting dictionary
dfs.keys()

dict_keys(['actor', 'address', 'category', 'city', 'country', 'customer', 'customer_list', 'film', 'film_actor', 'film_category', 'film_list', 'film_text', 'inventory', 'language', 'payment', 'rental', 'sales_by_film_category', 'sales_by_store', 'staff', 'staff_list', 'store'])

In [7]:
# each value for each key in the dictionary is a panda dataframe
type(dfs["actor"])

pandas.core.frame.DataFrame

In [8]:
# check the content of the df
dfs["actor"].head(3)

,actor_id,first_name,last_name,last_update
0,1.0,PENELOPE,GUINESS,2021-03-06 15:51:59
1,2.0,NICK,WAHLBERG,2021-03-06 15:51:59
2,3.0,ED,CHASE,2021-03-06 15:51:59


### register dfs to duckdb (tables) by joining dataframes

In [9]:
# duckdb.register("actor", dfs["actor"]) -one by one


## join the register tables

In [10]:
table_names = ["film", "film_actor", "film_category", "actor", "category"]

for table_name in table_names:
    duckdb.register(table_name, dfs[table_name])

duckdb.sql("DESC;").df()

,database,schema,name,column_names,column_types,temporary
0,temp,main,actor,"[actor_id, first_name, last_name, last_update]","[DOUBLE, VARCHAR, VARCHAR, TIMESTAMP]",True
1,temp,main,category,"[category_id, name, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",True
2,temp,main,film,"[film_id, title, description, release_year, la...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, BI...",True
3,temp,main,film_actor,"[actor_id, film_id, last_update]","[BIGINT, BIGINT, TIMESTAMP]",True
4,temp,main,film_category,"[film_id, category_id, last_update]","[BIGINT, BIGINT, TIMESTAMP]",True


## EDA with pandas
- a light introduction of pandas

## Visualization with pandas dataframe